## Домашняя работа по теме "Рекуррентные нейронные сети RNN LSTM GRU"

__Задание__

На вебинаре мы говорили что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для задачи сантимент анализа на данных с вебинара 
1. построить свёрточные архитектуры 
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN  и (RNN -> CNN)
4. сдлать выводы что получилось лучше

In [46]:
import pandas as pd
from sklearn.model_selection import train_test_split

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re


import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")
from nltk.probability import FreqDist

import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking, Flatten
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.callbacks import EarlyStopping  
from keras.losses import categorical_crossentropy
from sklearn.preprocessing import LabelEncoder

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Полина\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Загрузка данных

In [11]:
df = pd.read_excel('отзывы за лето.xls')
df.head(10)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
5,5,Всё удобно норм 👍👍👍,2017-08-14
6,5,Очень удобное приложение.,2017-08-14
7,5,Все устраивает,2017-08-14
8,5,У меня работает все четко. В отличии от банком...,2017-08-14
9,5,Очень все хорошо👍,2017-08-14


In [12]:
df_train, df_test = train_test_split(df, test_size=0.33, random_state=42)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_train.shape, df_test.shape

((13841, 3), (6818, 3))

Предобработка данных

In [13]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = txt.strip('\n').strip('\r').strip('\t')
    txt = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(txt))

    txt = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', txt)
    txt = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', txt)
    txt = re.sub(r'[\xad]|[\s+]', ' ', txt.strip())
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    
    return " ".join(txt)


In [14]:
df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)

In [15]:
# Готовим словарь
train_corpus = " ".join(df_train["Content"])
train_corpus = train_corpus.lower()
tokens = word_tokenize(train_corpus)

In [17]:
# Отфильтруем и возьмем только топ N токенов
tokens_filtered = [word for word in tokens if word.isalnum()] #фильтруем, чтобы остались только буквы и цифры

# Создаем словарь токен:количество в корпусе
dist = FreqDist(tokens_filtered)

In [18]:
max_words = 20000

tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер']

In [20]:
# Создаем словарь из топ-20000 токенов: токен:его рейтинг(место в топ-200)
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [21]:
def text_to_sequence(text, maxlen):

    """ Преобразуем трейновый и тестовый датасеты:
        - Приводим к нижнему регистру
        - Токенизируем
        Проверяем: если токен есть в словаре, то в результат, пишет его номер из словаря
        в противном случае - пропускаем, до полной длины добиваем нулями """
    
    result = []
    tokens = word_tokenize(text.lower()) 
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [22]:
max_len = 150

In [23]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["Content"]], dtype=np.int32)
x_train

array([[  0,   0,   0, ...,   1,   2,  15],
       [  0,   0,   0, ...,   0,   2, 177],
       [  0,   0,   0, ...,   0,   0,   5],
       ...,
       [  0,   0,   0, ..., 160,  27,  83],
       [  0,   0,   0, ...,   0,   0,  20],
       [  0,   0,   0, ...,   0, 111,   5]])

Для подсчета функции потерь y_train и y_test должны быть представлены one-hot кодированием.

In [24]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(df_train['Rating']) 
test_enc_labels = le.transform(df_test['Rating'])
le.classes_

array([1, 2, 3, 4, 5], dtype=int64)

In [25]:
num_classes = 5
y_train = tf.keras.utils.to_categorical(train_enc_labels, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(test_enc_labels, num_classes=num_classes)
y_train

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

### CNN

In [47]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  

epochs = 10
batch_size = 512

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
25/25 [==============================] - 9s 334ms/step - loss: 1.1860 - accuracy: 0.6895 - val_loss: 0.9693 - val_accuracy: 0.7011
Epoch 2/10
25/25 [==============================] - 15s 587ms/step - loss: 0.8480 - accuracy: 0.7169 - val_loss: 0.7892 - val_accuracy: 0.7350
Epoch 3/10
25/25 [==============================] - 15s 584ms/step - loss: 0.6955 - accuracy: 0.7697 - val_loss: 0.7266 - val_accuracy: 0.7451
Epoch 4/10
25/25 [==============================] - 12s 477ms/step - loss: 0.6233 - accuracy: 0.7888 - val_loss: 0.7047 - val_accuracy: 0.7545
Epoch 5/10
25/25 [==============================] - 8s 324ms/step - loss: 0.5729 - accuracy: 0.8088 - val_loss: 0.6989 - val_accuracy: 0.7545
Epoch 6/10
25/25 [==============================] - 9s 341ms/step - loss: 0.5327 - accuracy: 0.8243 - val_loss: 0.7036 - val_accuracy: 0.7545


In [48]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = pd.DataFrame([{'model': 'CNN', 'Test score': score[0], 'Test accuracy': score[1]}])

print('Test score:', score[0])
print('Test accuracy:', score[1])

14/14 [==============================] - 1s 68ms/step - loss: 0.6703 - accuracy: 0.7696
Test score: 0.6703131794929504
Test accuracy: 0.7695805430412292


### SimpleRNN

In [50]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(SimpleRNN(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


early_stopping=EarlyStopping(monitor='val_loss')  

epochs = 10
batch_size = 512

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
25/25 [==============================] - 18s 680ms/step - loss: 1.5019 - accuracy: 0.6006 - val_loss: 1.2810 - val_accuracy: 0.7199
Epoch 2/10
25/25 [==============================] - 16s 649ms/step - loss: 1.0278 - accuracy: 0.7401 - val_loss: 0.8602 - val_accuracy: 0.7321
Epoch 3/10
25/25 [==============================] - 15s 605ms/step - loss: 0.7431 - accuracy: 0.7722 - val_loss: 0.7686 - val_accuracy: 0.7451
Epoch 4/10
25/25 [==============================] - 13s 529ms/step - loss: 0.6585 - accuracy: 0.7838 - val_loss: 0.7502 - val_accuracy: 0.7480
Epoch 5/10
25/25 [==============================] - 13s 529ms/step - loss: 0.6055 - accuracy: 0.7958 - val_loss: 0.7590 - val_accuracy: 0.7509


In [51]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)

new_record = pd.DataFrame([{'model': 'RNN', 'Test score': score[0], 'Test accuracy': score[1]}])
metrics_df = pd.concat([metrics_df, new_record], ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

14/14 [==============================] - 2s 140ms/step - loss: 0.7350 - accuracy: 0.7728
Test score: 0.7350128889083862
Test accuracy: 0.7728073000907898


In [52]:
metrics_df

,model,Test score,Test accuracy
0,CNN,0.670313,0.769581
1,RNN,0.735013,0.772807


### LSTM

In [53]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(LSTM(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


early_stopping=EarlyStopping(monitor='val_loss')  

epochs = 10
batch_size = 512

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
25/25 [==============================] - 64s 2s/step - loss: 1.5508 - accuracy: 0.6659 - val_loss: 1.4382 - val_accuracy: 0.7141
Epoch 2/10
25/25 [==============================] - 67s 3s/step - loss: 1.1900 - accuracy: 0.7236 - val_loss: 0.9875 - val_accuracy: 0.7278
Epoch 3/10
25/25 [==============================] - 57s 2s/step - loss: 0.8157 - accuracy: 0.7573 - val_loss: 0.8181 - val_accuracy: 0.7495
Epoch 4/10
25/25 [==============================] - 61s 2s/step - loss: 0.6886 - accuracy: 0.7858 - val_loss: 0.7858 - val_accuracy: 0.7639
Epoch 5/10
25/25 [==============================] - 83s 3s/step - loss: 0.6113 - accuracy: 0.8105 - val_loss: 0.7770 - val_accuracy: 0.7610
Epoch 6/10
25/25 [==============================] - 77s 3s/step - loss: 0.5486 - accuracy: 0.8315 - val_loss: 0.7992 - val_accuracy: 0.7502


In [54]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)

new_record = pd.DataFrame([{'model': 'LSTM', 'Test score': score[0], 'Test accuracy': score[1]}])
metrics_df = pd.concat([metrics_df, new_record], ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

14/14 [==============================] - 11s 778ms/step - loss: 0.7702 - accuracy: 0.7653
Test score: 0.7701677083969116
Test accuracy: 0.7653270959854126


In [55]:
metrics_df

,model,Test score,Test accuracy
0,CNN,0.670313,0.769581
1,RNN,0.735013,0.772807
2,LSTM,0.770168,0.765327


### GRU

In [56]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(GRU(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


early_stopping=EarlyStopping(monitor='val_loss')  

epochs = 10
batch_size = 512

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
25/25 [==============================] - 46s 2s/step - loss: 1.5472 - accuracy: 0.6439 - val_loss: 1.4324 - val_accuracy: 0.7011
Epoch 2/10
25/25 [==============================] - 38s 2s/step - loss: 1.1765 - accuracy: 0.7058 - val_loss: 0.9970 - val_accuracy: 0.7004
Epoch 3/10
25/25 [==============================] - 37s 1s/step - loss: 0.8136 - accuracy: 0.7476 - val_loss: 0.7973 - val_accuracy: 0.7458
Epoch 4/10
25/25 [==============================] - 38s 2s/step - loss: 0.6704 - accuracy: 0.7797 - val_loss: 0.7799 - val_accuracy: 0.7451
Epoch 5/10
25/25 [==============================] - 39s 2s/step - loss: 0.5917 - accuracy: 0.8017 - val_loss: 0.7840 - val_accuracy: 0.7531


In [57]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)

new_record = pd.DataFrame([{'model': 'GRU', 'Test score': score[0], 'Test accuracy': score[1]}])
metrics_df = pd.concat([metrics_df, new_record], ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

14/14 [==============================] - 5s 334ms/step - loss: 0.7487 - accuracy: 0.7735
Test score: 0.7487202882766724
Test accuracy: 0.7735406160354614


### CNN -> LSTM

In [58]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(LSTM(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))


model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])


Epoch 1/10
25/25 [==============================] - 48s 2s/step - loss: 1.0934 - accuracy: 0.7017 - val_loss: 0.8396 - val_accuracy: 0.7076
Epoch 2/10
25/25 [==============================] - 44s 2s/step - loss: 0.7782 - accuracy: 0.7442 - val_loss: 0.7435 - val_accuracy: 0.7343
Epoch 3/10
25/25 [==============================] - 44s 2s/step - loss: 0.6725 - accuracy: 0.7697 - val_loss: 0.7187 - val_accuracy: 0.7401
Epoch 4/10
25/25 [==============================] - 44s 2s/step - loss: 0.5962 - accuracy: 0.7891 - val_loss: 0.7183 - val_accuracy: 0.7480
Epoch 5/10
25/25 [==============================] - 48s 2s/step - loss: 0.5438 - accuracy: 0.8115 - val_loss: 0.7344 - val_accuracy: 0.7458


In [59]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)

new_record = pd.DataFrame([{'model': 'CNN -> LSTM', 'Test score': score[0], 'Test accuracy': score[1]}])
metrics_df = pd.concat([metrics_df, new_record], ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

14/14 [==============================] - 8s 544ms/step - loss: 0.7075 - accuracy: 0.7656
Test score: 0.7075077891349792
Test accuracy: 0.7656204104423523


In [60]:
metrics_df

,model,Test score,Test accuracy
0,CNN,0.670313,0.769581
1,RNN,0.735013,0.772807
2,LSTM,0.770168,0.765327
3,GRU,0.748720,0.773541
4,CNN -> LSTM,0.707508,0.765620


### CNN -> GRU

In [61]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GRU(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
25/25 [==============================] - 44s 2s/step - loss: 1.1429 - accuracy: 0.6795 - val_loss: 0.8534 - val_accuracy: 0.7011
Epoch 2/10
25/25 [==============================] - 38s 2s/step - loss: 0.7908 - accuracy: 0.7035 - val_loss: 0.7774 - val_accuracy: 0.7069
Epoch 3/10
25/25 [==============================] - 38s 2s/step - loss: 0.7087 - accuracy: 0.7169 - val_loss: 0.7522 - val_accuracy: 0.7141
Epoch 4/10
25/25 [==============================] - 37s 1s/step - loss: 0.6343 - accuracy: 0.7640 - val_loss: 0.7440 - val_accuracy: 0.7394
Epoch 5/10
25/25 [==============================] - 43s 2s/step - loss: 0.5629 - accuracy: 0.7940 - val_loss: 0.7677 - val_accuracy: 0.7350


In [62]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)

new_record = pd.DataFrame([{'model': 'CNN -> GRU', 'Test score': score[0], 'Test accuracy': score[1]}])
metrics_df = pd.concat([metrics_df, new_record], ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

14/14 [==============================] - 6s 395ms/step - loss: 0.7530 - accuracy: 0.7539
Test score: 0.7530342936515808
Test accuracy: 0.7538867592811584


### CNN -> RNN

In [63]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(SimpleRNN(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
25/25 [==============================] - 19s 663ms/step - loss: 1.1230 - accuracy: 0.6680 - val_loss: 0.9000 - val_accuracy: 0.6924
Epoch 2/10
25/25 [==============================] - 20s 788ms/step - loss: 0.8183 - accuracy: 0.7296 - val_loss: 0.7682 - val_accuracy: 0.7336
Epoch 3/10
25/25 [==============================] - 18s 726ms/step - loss: 0.7003 - accuracy: 0.7662 - val_loss: 0.7284 - val_accuracy: 0.7357
Epoch 4/10
25/25 [==============================] - 17s 667ms/step - loss: 0.6182 - accuracy: 0.7839 - val_loss: 0.7019 - val_accuracy: 0.7466
Epoch 5/10
25/25 [==============================] - 17s 678ms/step - loss: 0.5576 - accuracy: 0.7958 - val_loss: 0.7058 - val_accuracy: 0.7502


In [64]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)

new_record = pd.DataFrame([{'model': 'CNN -> RNN', 'Test score': score[0], 'Test accuracy': score[1]}])
metrics_df = pd.concat([metrics_df, new_record], ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

14/14 [==============================] - 3s 197ms/step - loss: 0.6841 - accuracy: 0.7708
Test score: 0.6840823888778687
Test accuracy: 0.7707538604736328


### RNN -> CNN

In [65]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Activation("relu"))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
25/25 [==============================] - 32s 1s/step - loss: 1.1814 - accuracy: 0.6751 - val_loss: 0.8863 - val_accuracy: 0.7191
Epoch 2/10
25/25 [==============================] - 30s 1s/step - loss: 0.8132 - accuracy: 0.7418 - val_loss: 0.7651 - val_accuracy: 0.7386
Epoch 3/10
25/25 [==============================] - 28s 1s/step - loss: 0.6457 - accuracy: 0.7771 - val_loss: 0.7228 - val_accuracy: 0.7422
Epoch 4/10
25/25 [==============================] - 29s 1s/step - loss: 0.5340 - accuracy: 0.8076 - val_loss: 0.7403 - val_accuracy: 0.7386


In [66]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)

new_record = pd.DataFrame([{'model': 'RNN -> CNN', 'Test score': score[0], 'Test accuracy': score[1]}])
metrics_df = pd.concat([metrics_df, new_record], ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

14/14 [==============================] - 3s 234ms/step - loss: 0.7382 - accuracy: 0.7565
Test score: 0.7381527423858643
Test accuracy: 0.7565268278121948


### LSTM -> CNN

In [67]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(LSTM(128, return_sequences=True))
model.add(Activation("relu"))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
25/25 [==============================] - 77s 3s/step - loss: 1.2182 - accuracy: 0.6773 - val_loss: 0.9971 - val_accuracy: 0.7047
Epoch 2/10
25/25 [==============================] - 78s 3s/step - loss: 0.8754 - accuracy: 0.7282 - val_loss: 0.8033 - val_accuracy: 0.7357
Epoch 3/10
25/25 [==============================] - 79s 3s/step - loss: 0.7162 - accuracy: 0.7585 - val_loss: 0.7310 - val_accuracy: 0.7422
Epoch 4/10
25/25 [==============================] - 81s 3s/step - loss: 0.6166 - accuracy: 0.7770 - val_loss: 0.7330 - val_accuracy: 0.7415


In [68]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)

new_record = pd.DataFrame([{'model': 'LSTM -> CNN', 'Test score': score[0], 'Test accuracy': score[1]}])
metrics_df = pd.concat([metrics_df, new_record], ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

14/14 [==============================] - 11s 767ms/step - loss: 0.6932 - accuracy: 0.7656
Test score: 0.6931517124176025
Test accuracy: 0.7656204104423523


### GRU -> CNN

In [69]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(GRU(128, return_sequences=True))
model.add(Activation("relu"))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
25/25 [==============================] - 61s 2s/step - loss: 1.2448 - accuracy: 0.6826 - val_loss: 1.0112 - val_accuracy: 0.7083
Epoch 2/10
25/25 [==============================] - 50s 2s/step - loss: 0.8482 - accuracy: 0.7268 - val_loss: 0.7824 - val_accuracy: 0.7350
Epoch 3/10
25/25 [==============================] - 50s 2s/step - loss: 0.6718 - accuracy: 0.7600 - val_loss: 0.7322 - val_accuracy: 0.7451
Epoch 4/10
25/25 [==============================] - 52s 2s/step - loss: 0.5840 - accuracy: 0.7859 - val_loss: 0.7786 - val_accuracy: 0.7408


In [70]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)

new_record = pd.DataFrame([{'model': 'GRU -> CNN', 'Test score': score[0], 'Test accuracy': score[1]}])
metrics_df = pd.concat([metrics_df, new_record], ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

14/14 [==============================] - 7s 500ms/step - loss: 0.7451 - accuracy: 0.7552
Test score: 0.7450544238090515
Test accuracy: 0.755206823348999


### Подведение итогов

In [71]:
metrics_df.sort_values(by='Test score', ascending=False)

,model,Test score,Test accuracy
2,LSTM,0.770168,0.765327
5,CNN -> GRU,0.753034,0.753887
3,GRU,0.748720,0.773541
9,GRU -> CNN,0.745054,0.755207
7,RNN -> CNN,0.738153,0.756527
1,RNN,0.735013,0.772807
4,CNN -> LSTM,0.707508,0.765620
8,LSTM -> CNN,0.693152,0.765620
6,CNN -> RNN,0.684082,0.770754
0,CNN,0.670313,0.769581


In [74]:
metrics_df.sort_values(by='Test accuracy', ascending=False)

,model,Test score,Test accuracy
3,GRU,0.748720,0.773541
1,RNN,0.735013,0.772807
6,CNN -> RNN,0.684082,0.770754
0,CNN,0.670313,0.769581
4,CNN -> LSTM,0.707508,0.765620
8,LSTM -> CNN,0.693152,0.765620
2,LSTM,0.770168,0.765327
7,RNN -> CNN,0.738153,0.756527
9,GRU -> CNN,0.745054,0.755207
5,CNN -> GRU,0.753034,0.753887


В первой тройке по 2 критериям "Test score" и "Test accuracy" попадает модель GRU. Остальные модели показывают похожие результаты, но где-то "проседает" либо score, либо accuracy.